<a href="https://colab.research.google.com/github/lilnoes/notebooks/blob/main/translate%20en%20to%20fra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import os
import io
import unicodedata
import re
import tensorflow_datasets as tfds
tf.__version__

'2.2.0'

In [ ]:
def getsize(file):
  count = 1
  for i in file:
    count += 1
  return count

In [ ]:
ds_folder = "/content/drive/My Drive/datasets"

In [ ]:
path_to_zip = tf.keras.utils.get_file(
    "spa-eng.zip", origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True, cache_dir=ds_folder)

In [ ]:
filename = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"
filename

'/tmp/.keras/datasets/spa-eng/spa.txt'

In [ ]:
f = io.open(filename)

In [ ]:
getsize(f)

118965

In [ ]:
import unicodedata
def uni_to_asci(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s))

In [ ]:
import re
def preprocess_sentence(w):
  w = uni_to_asci(w.lower().strip())
  w = re.sub(r"([¿?.!,])", r" \1 ", w)
  w = re.sub(r'\s{2,}', " ", w)
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  return '<start> ' + w.strip() + ' <end>'

In [ ]:
def create_dataset(path, num):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  wp = [[preprocess_sentence(w) for w in line.split('\t')] for line in lines[:num]]
  return zip(*wp)

In [ ]:
asa = create_dataset(filename, 5)
en, sp = asa
en

('<start> go . <end>',
 '<start> go . <end>',
 '<start> go . <end>',
 '<start> go . <end>',
 '<start> hi . <end>')

In [ ]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)
  tensor = lang_tokenizer.texts_to_sequences(lang)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
  return tensor, lang_tokenizer

In [ ]:
def load_dataset(path, num):
  targ_lang, inp_lang = create_dataset(path, num)
  inp_tensor, inp_lang = tokenize(inp_lang)
  targ_tensor, targ_lang = tokenize(targ_lang)
  return inp_tensor, targ_tensor, inp_lang, targ_lang

In [ ]:
num = 3000
inp_tensor, targ_tensor, inp_lang, targ_lang = load_dataset(filename, num)

max_length_inp, max_length_targ = inp_tensor.shape[1], targ_tensor.shape[1]

inp_tensor_train, inp_tensor_val, targ_tensor_train, targ_tensor_val = train_test_split(inp_tensor, targ_tensor, test_size=0.2)

In [ ]:
BUFFER_SIZE = len(inp_tensor_train)
BATCH_SIZE = 32
steps_per_epoch = len(inp_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_targ_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((inp_tensor_train, targ_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True).cache()
for i in dataset:
  pass

In [ ]:
x, y = next(iter(dataset))
x.shape, y.shape

(TensorShape([32, 12]), TensorShape([32, 8]))

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_sz, emb_dim,enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_sz, emb_dim)
    self.gru = tf.keras.layers.GRU(enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

In [ ]:
hs = encoder.initialize_hidden_state()
eo, hs = encoder(x, hs)
eo.shape, hs.shape

(TensorShape([32, 12, 1024]), TensorShape([32, 1024]))

In [ ]:
class BAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    query = tf.expand_dims(query, 1)
    score = self.V(tf.nn.tanh(self.W1(query) + self.W2(values)))
    attn_weights = tf.nn.softmax(score, axis=1)
    context_vector = tf.reduce_sum(attn_weights*values, axis=1)
    return context_vector, attn_weights 

In [ ]:
battention = BAttention(units)

In [ ]:
ct, aw = battention(hs, eo)
ct.shape, aw.shape

(TensorShape([32, 1024]), TensorShape([32, 12, 1]))

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_sz, emb_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_sz, emb_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_sz)
    self.attention = BAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    context_vector, attention_weights = self.attention(hidden, enc_output)
    x = self.embedding(x)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
    output, state = self.gru(x)
    output = tf.reshape(output, (-1, output.shape[2]))
    x = self.fc(output)
    return x, state, attention_weights

In [ ]:
decoder = Decoder(vocab_targ_size, embedding_dim, units, BATCH_SIZE)
preds, state, attnw = decoder(tf.random.uniform((BATCH_SIZE,1)), hs, eo)
preds.shape, state.shape, attnw.shape

(TensorShape([32, 918]), TensorShape([32, 1024]), TensorShape([32, 12, 1]))

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss = loss_object(real, pred)
  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask
  return tf.reduce_mean(loss)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)
checkpoint_prefix

'./training_checkpoints/ckpt'

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)
    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']]*BATCH_SIZE, 1)

    for t in range(1, targ.shape[1]):
      pred, dec_hidden,_ = decoder(dec_input, dec_hidden, enc_output)
      loss += loss_function(targ[:,t], pred)
      dec_input = tf.expand_dims(targ[:,t], 1)

  batch_loss = loss/int(targ.shape[1])
  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return batch_loss

In [ ]:
import time
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()
  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for batch, (inp,targ) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 ==0:
      print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss.numpy():.4f}')

  if (epoch+1)%2==0:
    checkpoint.save(file_prefix=checkpoint_prefix)

  print(f'Epoch {epoch+1} Loss {total_loss/steps_per_epoch:.4f}')

  print(f'Time taken for 1 epoch {time.time()-start}')